# Odoo Debug & Diagnostic Toolkit

Comprehensive debugging tool for diagnosing and fixing Odoo issues safely without breaking the dashboard.

## Issues to Fix
1. **SCSS Variable Error**: Undefined variable `$mk-appsmenu-color` in muk_web_theme
2. **User Type Validation**: User assigned multiple user types (constraint violation)
3. **Service Startup**: Unable to load registry due to validation errors

## Step 1: Check Current Service Status

In [ ]:
import subprocess
import json

# Check if service is running
result = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22', 'systemctl', 'status', 'odoo-osusproperties.service', '--no-pager'],
    capture_output=True,
    text=True
)

print("=" * 60)
print("ODOO SERVICE STATUS")
print("=" * 60)
print(result.stdout[:1000])

if 'running' in result.stdout:
    print("\n✅ Service is RUNNING")
elif 'failed' in result.stdout or 'inactive' in result.stdout:
    print("\n❌ Service is FAILED/INACTIVE")
else:
    print("\n⚠️  Service status UNKNOWN")

## Step 2: Analyze Recent Log Errors

In [ ]:
# Get recent errors from logs
result = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22', 'tail', '-n', '150', '/var/odoo/osusproperties/logs/odoo-server.log'],
    capture_output=True,
    text=True
)

print("=" * 60)
print("RECENT LOG ENTRIES (Last 150 lines)")
print("=" * 60)
print(result.stdout)

# Extract error summary
lines = result.stdout.split('\n')
errors = [line for line in lines if 'ERROR' in line or 'CRITICAL' in line]
warnings = [line for line in lines if 'WARNING' in line]

print("\n" + "=" * 60)
print(f"ERRORS: {len(errors)}  |  WARNINGS: {len(warnings)}")
print("=" * 60)

## Step 3: Check SCSS Variable Issue (muk_web_theme)

In [ ]:
# Find muk_web_theme SCSS files
result = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22', 
     'find /var/odoo/osusproperties/extra-addons -name "*.scss" -path "*muk_web_theme*" 2>/dev/null'],
    capture_output=True,
    text=True
)

print("=" * 60)
print("MUK_WEB_THEME SCSS FILES")
print("=" * 60)
print(result.stdout)

# Check if the variable is defined
result2 = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22',
     'grep -r "mk-appsmenu-color" /var/odoo/osusproperties/extra-addons/*/static/src/scss/ 2>/dev/null || echo "NOT FOUND"'],
    capture_output=True,
    text=True
)

print("\n" + "=" * 60)
print("SEARCHING FOR: $mk-appsmenu-color VARIABLE")
print("=" * 60)
print(result2.stdout if result2.stdout else "Variable definition not found")

## Step 4: Check User Type Validation Issue

In [ ]:
# Check for users with multiple types
result = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22',
     'sudo', '-u', 'odoo', 'psql', '-d', 'osusproperties', '-c',
     'SELECT id, login, COUNT(*) as type_count FROM res_users ru LEFT JOIN res_groups_users_rel rgu ON ru.id = rgu.uid WHERE rgu.gid IN (SELECT id FROM res_groups WHERE "is_portaluser"=true OR "is_publicuser"=true) GROUP BY ru.id, ru.login HAVING COUNT(*) > 1'],
    capture_output=True,
    text=True
)

print("=" * 60)
print("USERS WITH MULTIPLE USER TYPES")
print("=" * 60)
if 'id' in result.stdout:
    print(result.stdout)
else:
    print("Checking via direct SQL query...")
    # Try simpler check
    result2 = subprocess.run(
        ['ssh', 'root@139.84.163.11', '-p', '22',
         'sudo -u odoo psql -d osusproperties << EOSQL\nSELECT COUNT(*) FROM res_users WHERE id > 0;\nEOSQL'],
        capture_output=True,
        text=True
    )
    print(f"Total users: {result2.stdout}")

## Step 5: SOLUTION - Fix SCSS Variable

In [ ]:
print("=" * 60)
print("SOLUTION 1: Add Missing SCSS Variable")
print("=" * 60)
print("""
The error 'Undefined variable: "$mk-appsmenu-color"' occurs because:
1. muk_web_theme uses this variable
2. It's not defined in the primary_variables.scss
3. Bootstrap SCSS compiler fails

FIX OPTIONS:

Option A: Add the variable to muk_web_theme's variables.scss
Option B: Disable muk_web_theme temporarily
Option C: Use --no-assets flag to skip asset compilation

RECOMMENDED: Option A (Add variable definition)
""")

In [ ]:
# Find muk_web_theme variables file
result = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22',
     'find /var/odoo/osusproperties/extra-addons -name "variables.scss" -path "*muk_web_theme*" | head -n 1'],
    capture_output=True,
    text=True
)

variables_file = result.stdout.strip()
print(f"Variables file: {variables_file}")

if variables_file:
    # Read the file to see current content
    result = subprocess.run(
        ['ssh', 'root@139.84.163.11', '-p', '22', f'head -n 30 {variables_file}'],
        capture_output=True,
        text=True
    )
    print(f"\nCurrent content (first 30 lines):\n{result.stdout}")

## Step 6: QUICK FIX - Restart Without Asset Compilation

In [ ]:
print("=" * 60)
print("QUICK FIX: Restart Service (Skip Asset Issues)")
print("=" * 60)
print("""
To get the service running immediately:

1. Stop the service
2. Start without triggering asset recompilation
3. Fix database issues (user types)
4. Then recompile assets

Command to try:
""")

command = "ssh root@139.84.163.11 \"systemctl start odoo-osusproperties.service\""
print(f"\n$ {command}")

print("\nThis will:\n✓ Start the service\n✓ Use previously compiled assets\n✓ Skip the SCSS compilation error\n✓ Still fix the dashboard JavaScript")

## Step 7: Fix Database User Type Issue

In [ ]:
print("=" * 60)
print("DATABASE FIX: Resolve User Type Validation Error")
print("=" * 60)
print("""
The error 'The user cannot have more than one user types' means:
- A user is assigned to both portal and public groups (or similar)
- Odoo 17 constraint: users can only be ONE type

FIX STEPS:

1. Connect to database
2. Find users in multiple user type groups
3. Remove them from unnecessary groups
4. Verify fix

SQL to find problematic users:
""")

sql_query = """
SELECT 
    u.id, 
    u.login, 
    STRING_AGG(g.name, ', ') as groups
FROM res_users u
JOIN res_groups_users_rel gu ON u.id = gu.uid
JOIN res_groups g ON gu.gid = g.id
WHERE g.name IN ('Portal Users', 'Public User', 'base.group_portal', 'base.group_public')
GROUP BY u.id, u.login
HAVING COUNT(DISTINCT g.category_id) > 1;
"""

print(sql_query)

## Step 8: Recommended Action Plan

In [ ]:
print("=" * 60)
print("RECOMMENDED ACTION PLAN")
print("=" * 60)
print("""
╔════════════════════════════════════════════════════════════╗
║                PHASE 1: IMMEDIATE RECOVERY                  ║
╚════════════════════════════════════════════════════════════╝

GOAL: Get service running with working dashboard
TIME: ~2 minutes

1. ✓ Start service (uses cached assets, skips compilation)
   ssh root@139.84.163.11 "systemctl start odoo-osusproperties.service"
   
2. ✓ Wait 5 seconds for startup
   ssh root@139.84.163.11 "sleep 5"
   
3. ✓ Verify dashboard works
   curl http://localhost:8070/web
   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╔════════════════════════════════════════════════════════════╗
║               PHASE 2: FIX DATABASE ISSUES                 ║
╚════════════════════════════════════════════════════════════╝

GOAL: Fix user type constraint violation
TIME: ~1 minute

1. ✓ Find problematic users with SQL query
2. ✓ Remove them from extra user type groups
3. ✓ Verify constraint is satisfied
   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╔════════════════════════════════════════════════════════════╗
║            PHASE 3: RECOMPILE ASSETS (Optional)            ║
╚════════════════════════════════════════════════════════════╝

GOAL: Fresh asset compilation (only if needed)
TIME: ~2 minutes

1. ✓ Fix SCSS variable in muk_web_theme
2. ✓ Recompile web module
3. ✓ Verify no SCSS errors

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

KEY POINT: The Synconics dashboard fix IS ALREADY APPLIED
to the source code. We just need to get the service running!
""")

## Step 9: Execute Phase 1 - Recovery

In [ ]:
import time

print("EXECUTING PHASE 1: SERVICE RECOVERY...\n")

# Step 1: Start service
print("[1/3] Starting Odoo service...")
result = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22', 'systemctl', 'start', 'odoo-osusproperties.service'],
    capture_output=True,
    text=True
)
print("✓ Start command sent")

# Step 2: Wait for startup
print("\n[2/3] Waiting for service to initialize (5 seconds)...")
for i in range(5):
    print(f"  {5-i}...", end='', flush=True)
    time.sleep(1)
print("\n✓ Wait complete")

# Step 3: Verify status
print("\n[3/3] Checking service status...")
result = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22', 'systemctl', 'status', 'odoo-osusproperties.service', '--no-pager'],
    capture_output=True,
    text=True
)

if 'running' in result.stdout or 'active' in result.stdout:
    print("✅ SERVICE IS RUNNING!")
    print("\nYou can now access the dashboard at:")
    print("  → https://erp.erposus.com/web")
else:
    print("⚠️  Service may still be initializing...")
    print(result.stdout[-500:])

## Step 10: Verify Dashboard Health

In [ ]:
print("=" * 60)
print("VERIFY DASHBOARD FUNCTIONALITY")
print("=" * 60)

# Test 1: Can we reach the web interface?
result = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22', 'curl', '-s', '-o', '/dev/null', '-w', '%{http_code}', 'http://localhost:8070/web'],
    capture_output=True,
    text=True,
    timeout=5
)

http_code = result.stdout.strip()
print(f"\n1. Web Interface HTTP Code: {http_code}")

if http_code == '200':
    print("   ✅ Web interface is accessible")
elif http_code == '303':
    print("   ✅ Web interface responds (redirect, expected)")
else:
    print(f"   ⚠️  Unexpected response code: {http_code}")

# Test 2: Check for JavaScript errors
print("\n2. Check Recent Logs for JavaScript Errors...")
result = subprocess.run(
    ['ssh', 'root@139.84.163.11', '-p', '22',
     'tail -n 30 /var/odoo/osusproperties/logs/odoo-server.log | grep -i "error\|cannot read"'],
    capture_output=True,
    text=True
)

if result.stdout.strip():
    print(f"   ⚠️  Found errors:\n{result.stdout}")
else:
    print("   ✅ No critical JavaScript errors in logs")

print("\n" + "=" * 60)
print("NEXT STEPS:")
print("=" * 60)
print("""
1. Open your browser and go to: https://erp.erposus.com/web
2. Navigate to: Sales & Invoicing Dashboard
3. You should see the dashboard WITHOUT the JavaScript error
4. If you still see an error, run Step 11 (Database Fix)
""")

## Step 11: Fix Database User Type Issue (if needed)

In [ ]:
print("=" * 60)
print("DATABASE FIX: User Type Constraint Violation")
print("=" * 60)
print("""
If the service won't start due to 'user cannot have more than one user types':

The fix requires:
1. Identifying users in conflicting groups
2. Removing them from secondary user type groups
3. Verifying Odoo constraint is satisfied
""")

# Create fix script
fix_script = """
import psycopg2
import sys

try:
    conn = psycopg2.connect(
        dbname='osusproperties',
        user='odoo',
        password='odoo',
        host='localhost'
    )
    cur = conn.cursor()
    
    # Find problematic users
    cur.execute("""
        SELECT u.id, u.login, array_agg(g.name)
        FROM res_users u
        JOIN res_groups_users_rel gu ON u.id = gu.uid
        JOIN res_groups g ON gu.gid = g.id
        WHERE g.is_portaluser OR g.is_publicuser
        GROUP BY u.id, u.login
        HAVING COUNT(DISTINCT CASE WHEN g.is_portaluser OR g.is_publicuser THEN 1 END) > 1
    """)
    
    problematic = cur.fetchall()
    
    if problematic:
        print(f"Found {len(problematic)} users with conflicting user types:")
        for user_id, login, groups in problematic:
            print(f"  - {login} (ID: {user_id}): {groups}")
            print(f"    → Removing from secondary groups...")
            # Keep only the first user type group
            cur.execute("""
                DELETE FROM res_groups_users_rel
                WHERE uid = %s AND gid IN (
                    SELECT id FROM res_groups 
                    WHERE is_portaluser OR is_publicuser
                ) AND gid NOT IN (
                    SELECT id FROM res_groups
                    WHERE is_portaluser OR is_publicuser
                    LIMIT 1
                )
            """, (user_id,))
        
        conn.commit()
        print(f"\n✅ Fixed {len(problematic)} users")
    else:
        print("✅ No conflicting users found")
    
    cur.close()
    conn.close()
except Exception as e:
    print(f"❌ Error: {e}")
    sys.exit(1)
"""

print("\nDatabase fix script created. To run it:")
print("""echo 'PYTHON_SCRIPT' | ssh root@139.84.163.11 'python3'""")